In [123]:
import numpy as np 
import math 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import scipy.integrate as integrate

In [124]:
data_1 = {
    'Maturity': [0, 1, 3, 5, 7, 10],
    'CDS_rate': [np.nan, 100, 110, 120, 120, 125]
}
cds_rate = pd.DataFrame(data_1)
cds_rate

,Maturity,CDS_rate
0,0,NaN
1,1,100.0
2,3,110.0
3,5,120.0
4,7,120.0
5,10,125.0


# Question 1

In [135]:
def average_hazard_rate(cds_rate):
    return cds_rate / LGD / 10000

In [136]:
def survival_prob(maturity, average_hazard_rate):
    return np.exp(-average_hazard_rate * maturity)

In [137]:
LGD = 0.4
h_rate = []

for i in range(cds_rate.shape[0]):
    maturity = cds_rate['Maturity'].iloc[i] 
    cds_rate_value = cds_rate['CDS_rate'].iloc[i]  
    average_hazard_rate_value = average_hazard_rate(cds_rate_value)
    h_rate.append({'Maturity': maturity, 'Average lamda': average_hazard_rate_value})

h_rate_df = pd.DataFrame(h_rate)
h_rate_df

,Maturity,Average lamda
0,0,NaN
1,1,0.02500
2,3,0.02750
3,5,0.03000
4,7,0.03000
5,10,0.03125


In [138]:
h_rate_df['Q(t>T)'] = survival_prob(h_rate_df['Maturity'], h_rate_df['Average lamda'])
h_rate_df.iloc[0, 2] = 1

In [139]:
h_rate_df['fwd_h_rate'] = 0
for i in range(1, h_rate_df.shape[0]):
    h_rate_df.loc[i, 'fwd_h_rate'] = -np.log(h_rate_df.loc[i, 'Q(t>T)'] / h_rate_df.loc[i - 1, 'Q(t>T)'])\
    / (h_rate_df.loc[i, 'Maturity'] - h_rate_df.loc[i - 1, 'Maturity'])       

In [141]:
h_rate_df['default_prob'] = 0
for i in range(1, h_rate_df.shape[0]):
    h_rate_df.loc[i, 'default_prob'] = h_rate_df.loc[i - 1, 'Q(t>T)'] - h_rate_df.loc[i, 'Q(t>T)']


In [142]:
h_rate_df

,Maturity,Average lamda,Q(t>T),fwd_h_rate,default_prob
0,0,NaN,1.000000,0.000000,0.000000
1,1,0.02500,0.975310,0.025000,0.024690
2,3,0.02750,0.920811,0.028750,0.054498
3,5,0.03000,0.860708,0.033750,0.060103
4,7,0.03000,0.810584,0.030000,0.050124
5,10,0.03125,0.731616,0.034167,0.078969


# Question 2